In [0]:
%scala
package org.apache.spark.sql.hive.client

object HiveVersionUtil {
  def hiveVersion(version: String): HiveVersion = {
    IsolatedClientLoader.hiveVersion(version)
  }
}

Warning: classes defined within packages cannot be redefined without a cluster restart.
Compilation successful.

In [0]:
%scala
package org.apache.spark.sql.hive.client

import java.io.File
import java.lang.reflect.InvocationTargetException
import java.net.{URL, URLClassLoader}
import java.util

import scala.util.Try

import org.apache.commons.io.{FileUtils, IOUtils}
import org.apache.hadoop.conf.Configuration

import org.apache.spark.SparkConf
import org.apache.spark.deploy.SparkSubmitUtils
import org.apache.spark.internal.Logging
import org.apache.spark.sql.catalyst.util.quietly
import org.apache.spark.sql.hive.HiveUtils
import org.apache.spark.sql.internal.NonClosableMutableURLClassLoader
import org.apache.spark.util.{MutableURLClassLoader, Utils}

object HiveMetastoreJarDownloadUtil {
  def downloadVersion(
      version: HiveVersion,
      hadoopVersion: String = "2.7.3",
      ivyPath: Option[String] = None): Seq[URL] = {
    val hiveArtifacts = version.extraDeps ++
      Seq("hive-metastore", "hive-exec", "hive-common", "hive-serde")
        .map(a => s"org.apache.hive:$a:${version.fullVersion}") ++
      Seq("com.google.guava:guava:14.0.1",
        s"org.apache.hadoop:hadoop-client:$hadoopVersion")

    val classpath = SparkSubmitUtils.resolveMavenCoordinates(
        hiveArtifacts.mkString(","),
        SparkSubmitUtils.buildIvySettings(
          Some("http://www.datanucleus.org/downloads/maven2"),
          ivyPath),
        exclusions = version.exclusions)
    val allFiles = classpath.split(",").map(new File(_)).toSet

    // TODO: Remove copy logic.
    val tempDir = Utils.createTempDir(namePrefix = s"hive-${version}")
    allFiles.foreach(f => FileUtils.copyFileToDirectory(f, tempDir))
    println(s"Downloaded metastore jars to ${tempDir.getCanonicalPath}")
    tempDir.listFiles().map(_.toURI.toURL)
  }
}

Warning: classes defined within packages cannot be redefined without a cluster restart.
Compilation successful.

In [0]:
%scala
val hive_v = org.apache.spark.sql.hive.client.HiveVersionUtil.hiveVersion("3.1.0")

hive_v_1_1: org.apache.spark.sql.hive.client.HiveVersion = v3_1

In [0]:
%scala
org.apache.spark.sql.hive.client.HiveMetastoreJarDownloadUtil.downloadVersion(hive_v)

http://www.datanucleus.org/downloads/maven2 added as a remote repository with the name: repo-1
:: loading settings :: url = jar:file:/databricks/jars/----workspace_spark_3_1--maven-trees--hive-2.3__hadoop-2.7--org.apache.ivy--ivy--org.apache.ivy__ivy__2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.logging.log4j#log4j-api added as a dependency
org.apache.derby#derby added as a dependency
org.apache.hive#hive-metastore added as a dependency
org.apache.hive#hive-exec added as a dependency
org.apache.hive#hive-common added as a dependency
org.apache.hive#hive-serde added as a dependency
com.google.guava#guava added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f324d0df-7084-4e30-af85-a63ff9c60865;1.0
	confs: [default]
	found org.apache.logging.log4j#log4j-api;2.10.0 in central
	found org.apache.derby#derby;10.14.1.0 in central
	found org.apache.hive#hive-metastore;3.1.2 in central
	found org.apache.hive#hive-serde;3.1.2 in central
	found org.apache.hive#hive-common;3.1.2 in central
	found org.apache.hive#hive-classification;3.1.2 in central
	found org.slf4j#slf4j-api;1.7.10 in central
	found org.apache.hive#hive-upgrade-acid;3.1.2 in central
	found org.apache.hive#hive-shims;3.1.2 in central
	found org.apache.hive.shims#hive-shims-common;3.1.2 in central
	found org.apache.logging.log4j#log4j-slf4j-impl;2.10.0 in central
	found com.google.guava#guava;19.0 in central
	found commons-lang#commons-lang;2.6 in central
	found org.apache.thrift#libthrift;0.9.3 in central
	found org.apache.httpcomponents#httpclient;4.5.2 in central
	found org.apache.httpcomponents#httpcore;4.4.4 in central
	found commons-logging#commons-logging;1.2 in central
	found commons-codec#commons-codec;1.7 in central
	found org.apache.zookeeper#zookeeper;3.4.6 in central
	found org.slf4j#slf4j-log4j12;1.6.1 in central
	found log4j#log4j;1.2.16 in central
	found jline#jline;2.12 in central
	found io.netty#netty;3.7.0.Final in central
	found org.apache.logging.log4j#log4j-core;2.10.0 in central
	found org.apache.hive.shims#hive-shims-0.23;3.1.2 in central
	found org.apache.hadoop#hadoop-yarn-server-resourcemanager;3.1.0 in central
	found javax.servlet#javax.servlet-api;3.1.0 in central
	found org.apache.hadoop#hadoop-annotations;3.1.0 in central
	found com.google.inject.extensions#guice-servlet;4.0 in central
	found com.google.inject#guice;4.0 in central
	found javax.inject#javax.inject;1 in central
	found aopalliance#aopalliance;1.0 in central
	found com.google.protobuf#protobuf-java;2.5.0 in central
	found commons-io#commons-io;2.4 in central
	found com.sun.jersey#jersey-json;1.19 in central
	found org.codehaus.jettison#jettison;1.1 in central
	found com.sun.xml.bind#jaxb-impl;2.2.3-1 in central
	found javax.xml.bind#jaxb-api;2.2.11 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
	found org.codehaus.jackson#jackson-mapper-asl;1.9.13 in central
	found org.codehaus.jackson#jackson-jaxrs;1.9.13 in central
	found org.codehaus.jackson#jackson-xc;1.9.13 in central
	found com.sun.jersey.contribs#jersey-guice;1.19 in central
	found org.apache.hadoop#hadoop-yarn-common;3.1.0 in central
	found org.apache.hadoop#hadoop-yarn-api;3.1.0 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.9.5 in central
	found org.apache.hadoop#hadoop-auth;3.1.0 in central
	found com.nimbusds#nimbus-jose-jwt;4.41.1 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found net.minidev#json-smart;2.3 in central
	found net.minidev#accessors-smart;1.2 in central
	found org.ow2.asm#asm;5.0.4 in central
	found org.apache.kerby#kerb-simplekdc;1.0.1 in central
	found org.apache.kerby#kerb-client;1.0.1 in central
	found org.apache.kerby#kerby-config;1.0.1 in central
	found org.apache.kerby#kerb-core;1.0.1 in central
	found org.apache.k

In [0]:
%sh ls /local_disk0/tmp/hive-v3_1-2f41c186-61e4-4ad8-9e99-2f137edae0d3

aopalliance_aopalliance-1.0.jar
co.cask.tephra_tephra-api-0.6.0.jar
co.cask.tephra_tephra-core-0.6.0.jar
co.cask.tephra_tephra-hbase-compat-1.0-0.6.0.jar
com.carrotsearch_hppc-0.7.2.jar
com.cedarsoftware_java-util-1.9.0.jar
com.cedarsoftware_json-io-2.5.1.jar
com.esri.geometry_esri-geometry-api-2.0.0.jar
com.fasterxml.jackson.core_jackson-annotations-2.9.5.jar
com.fasterxml.jackson.core_jackson-core-2.9.5.jar
com.fasterxml.jackson.core_jackson-databind-2.9.5.jar
com.fasterxml.jackson.jaxrs_jackson-jaxrs-base-2.9.5.jar
com.fasterxml.jackson.jaxrs_jackson-jaxrs-json-provider-2.9.5.jar
com.fasterxml.jackson.module_jackson-module-jaxb-annotations-2.9.5.jar
com.fasterxml.woodstox_woodstox-core-5.0.3.jar
com.github.joshelser_dropwizard-metrics-hadoop-metrics2-reporter-0.1.2.jar
com.github.stephenc.jcip_jcip-annotations-1.0-1.jar
com.google.code.findbugs_jsr305-3.0.1.jar
com.google.code.gson_gson-2.2.4.jar
com.google.guava_guava-19.0.jar
com.google.inject.extensions_guice-assistedinject-3.0.jar
com.google.inject.extensions_guice-servlet-4.0.jar
com.google.inject_guice-4.0.jar
com.google.protobuf_protobuf-java-2.5.0.jar
com.google.re2j_re2j-1.1.jar
com.jcraft_jsch-0.1.54.jar
com.jolbox_bonecp-0.8.0.RELEASE.jar
com.microsoft.sqlserver_mssql-jdbc-6.2.1.jre7.jar
com.nimbusds_nimbus-jose-jwt-4.41.1.jar
com.sun.jersey.contribs_jersey-guice-1.19.jar
com.sun.jersey_jersey-client-1.19.jar
com.sun.jersey_jersey-core-1.19.jar
com.sun.jersey_jersey-json-1.19.jar
com.sun.jersey_jersey-server-1.19.jar
com.sun.jersey_jersey-servlet-1.19.jar
com.sun.xml.bind_jaxb-impl-2.2.3-1.jar
com.tdunning_json-1.8.jar
com.thoughtworks.paranamer_paranamer-2.3.jar
com.vlkan_flatbuffers-1.2.0-3f79e055.jar
com.yahoo.datasketches_memory-0.9.0.jar
com.yahoo.datasketches_sketches-core-0.9.0.jar
com.zaxxer_HikariCP-2.6.1.jar
com.zaxxer_HikariCP-java7-2.4.12.jar
commons-beanutils_commons-beanutils-1.9.3.jar
commons-cli_commons-cli-1.2.jar
commons-codec_commons-codec-1.7.jar
commons-collections_commons-collections-3.2.2.jar
commons-daemon_commons-daemon-1.0.13.jar
commons-dbcp_commons-dbcp-1.4.jar
commons-io_commons-io-2.4.jar
commons-lang_commons-lang-2.6.jar
commons-logging_commons-logging-1.2.jar
commons-net_commons-net-3.6.jar
commons-pool_commons-pool-1.5.4.jar
de.ruedigermoeller_fst-2.50.jar
dnsjava_dnsjava-2.1.7.jar
io.airlift_aircompressor-0.10.jar
io.dropwizard.metrics_metrics-core-3.2.1.jar
io.dropwizard.metrics_metrics-json-3.1.0.jar
io.dropwizard.metrics_metrics-jvm-3.1.0.jar
io.netty_netty-3.7.0.Final.jar
io.netty_netty-all-4.1.17.Final.jar
io.netty_netty-buffer-4.1.17.Final.jar
io.netty_netty-common-4.1.17.Final.jar
it.unimi.dsi_fastutil-6.5.6.jar
javax.inject_javax.inject-1.jar
javax.jdo_jdo-api-3.0.1.jar
javax.servlet.jsp_jsp-api-2.1.jar
javax.servlet_javax.servlet-api-3.1.0.jar
javax.transaction_jta-1.1.jar
javax.transaction_transaction-api-1.1.jar
javax.ws.rs_jsr311-api-1.1.1.jar
javax.xml.bind_jaxb-api-2.2.11.jar
javolution_javolution-5.5.1.jar
jline_jline-2.12.jar
joda-time_joda-time-2.9.9.jar
junit_junit-4.11.jar
log4j_log4j-1.2.17.jar
net.minidev_accessors-smart-1.2.jar
net.minidev_json-smart-2.3.jar
net.sf.jpam_jpam-1.1.jar
net.sf.opencsv_opencsv-2.3.jar
org.antlr_ST4-4.0.4.jar
org.antlr_antlr-runtime-3.5.2.jar
org.apache.ant_ant-1.9.1.jar
org.apache.ant_ant-launcher-1.9.1.jar
org.apache.arrow_arrow-format-0.8.0.jar
org.apache.arrow_arrow-memory-0.8.0.jar
org.apache.arrow_arrow-vector-0.8.0.jar
org.apache.avro_avro-1.7.7.jar
org.apache.calcite_calcite-core-1.16.0.jar
org.apache.calcite_calcite-linq4j-1.16.0.jar
org.apache.commons_commons-compress-1.9.jar
org.apache.commons_commons-configuration2-2.1.1.jar
org.apache.commons_commons-crypto-1.0.0.jar
org.apache.commons_commons-lang3-3.6.jar
org.apache.commons_commons-math3-3.1.1.jar
org.apache.derby_derby-10.14.1.0.jar
org.apache.geronimo.specs_geronimo-jcache_1.0_spec-1.0-alpha-1.jar
org.apache.hadoop_hadoop-annotations-3.1.0.jar
org.apache.hadoop_hadoop-auth-3.1.0.jar
org.apache.hadoop_hadoop-client

In [0]:
%sh mkdir /dbfs/metastore_jars/

In [0]:
%sh mkdir /dbfs/metastore_jars/hive-v3_1

In [0]:
%sh cp -r /local_disk0/tmp/hive-v3_1-2f41c186-61e4-4ad8-9e99-2f137edae0d3/* /dbfs/metastore_jars/hive-v3_1

In [0]:
%scala
// jars are in dbfs
display(dbutils.fs.ls("/metastore_jars/hive-v3_1"))

path,name,size
dbfs:/metastore_jars/hive-v3_1/aopalliance_aopalliance-1.0.jar,aopalliance_aopalliance-1.0.jar,4467
dbfs:/metastore_jars/hive-v3_1/co.cask.tephra_tephra-api-0.6.0.jar,co.cask.tephra_tephra-api-0.6.0.jar,8836
dbfs:/metastore_jars/hive-v3_1/co.cask.tephra_tephra-core-0.6.0.jar,co.cask.tephra_tephra-core-0.6.0.jar,635211
dbfs:/metastore_jars/hive-v3_1/co.cask.tephra_tephra-hbase-compat-1.0-0.6.0.jar,co.cask.tephra_tephra-hbase-compat-1.0-0.6.0.jar,28216
dbfs:/metastore_jars/hive-v3_1/com.carrotsearch_hppc-0.7.2.jar,com.carrotsearch_hppc-0.7.2.jar,1671083
dbfs:/metastore_jars/hive-v3_1/com.cedarsoftware_java-util-1.9.0.jar,com.cedarsoftware_java-util-1.9.0.jar,58487
dbfs:/metastore_jars/hive-v3_1/com.cedarsoftware_json-io-2.5.1.jar,com.cedarsoftware_json-io-2.5.1.jar,75232
dbfs:/metastore_jars/hive-v3_1/com.esri.geometry_esri-geometry-api-2.0.0.jar,com.esri.geometry_esri-geometry-api-2.0.0.jar,914317
dbfs:/metastore_jars/hive-v3_1/com.fasterxml.jackson.core_jackson-annotations-2.9.5.jar,com.fasterxml.jackson.core_jackson-annotations-2.9.5.jar,66983
dbfs:/metastore_jars/hive-v3_1/com.fasterxml.jackson.core_jackson-core-2.9.5.jar,com.fasterxml.jackson.core_jackson-core-2.9.5.jar,321590
